In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sha2, concat_ws
from pyspark.sql.functions import *
spark  = SparkSession.builder\
                    .appName("lendingClubProject")\
                    .config('spark.shuffle.useOldFetchProtocol', 'true') \
                    .config("spark.sql.warehouse.dir","/user/anil/warehouse")\
                    .enableHiveSupport()\
                    .master('yarn')\
                    .getOrCreate()

# Creating external hive tables on top of cleansed files

In [2]:
customers_df = spark.read \
.format("parquet") \
.load("/user/anil/lendingClubProject/cleansed/customersParquet")

In [3]:
customers_df

member_id,emp_title,emp_length,home_ownership,annual_income,address_state,address_zipcode,address_country,grade,sub_grade,verification_status,total_high_credit_limit,application_type,join_annual_income,verification_status_joint,Ingeste_date
5bad76b2473e8638c...,Funeral Director,3,RENT,56000.0,VA,221xx,USA,C,C2,Verified,61768.0,Individual,null,null,2024-11-13 08:02:04
2651c8e4bc69338d8...,Technician,3,OWN,120000.0,OK,730xx,USA,A,A4,Not Verified,261126.0,Individual,null,null,2024-11-13 08:02:04
2b3f2a8caa61c35eb...,International Fli...,10,MORTGAGE,60000.0,TX,773xx,USA,C,C3,Not Verified,144189.0,Individual,null,null,2024-11-13 08:02:04
432ea84f5b9715a27...,Carpenter,5,RENT,12000.0,CA,945xx,USA,A,A5,Not Verified,16300.0,Individual,null,null,2024-11-13 08:02:04
cfbb2e6dd37657fcc...,Office work,8,MORTGAGE,30000.0,FL,334xx,USA,A,A3,Not Verified,43000.0,Individual,null,null,2024-11-13 08:02:04
d1946d1099882c06a...,Deli Manager,10,MORTGAGE,60000.0,NY,119xx,USA,B,B3,Not Verified,305570.0,Individual,null,null,2024-11-13 08:02:04
3a8b3e793b034aa4b...,Hair stylist,1,RENT,30000.0,GA,303xx,USA,B,B3,Not Verified,56962.0,Individual,null,null,2024-11-13 08:02:04
3576f3b052b25e27b...,RECEPTIONIST,10,RENT,102000.0,NJ,070xx,USA,B,B3,Verified,35435.0,Individual,null,null,2024-11-13 08:02:04
a41b62fdeb37c6783...,Office Administrator,5,MORTGAGE,70000.0,FL,330xx,USA,D,D4,Source Verified,283460.0,Individual,null,null,2024-11-13 08:02:04
efc9537c442236da0...,Pharmacist,1,RENT,80000.0,TN,379xx,USA,A,A5,Not Verified,64936.0,Individual,null,null,2024-11-13 08:02:04


In [4]:
spark.sql('create database if not exists lendingClub_525')

""


In [2]:
spark.sql('use lendingClub_525')

""


In [10]:
spark.sql("""create table if not exists lendingClub_525.customers (member_id string, emp_title string, emp_length int, 
home_ownership string, annual_income float, address_state string, address_zipcode string, address_country string, grade string, 
sub_grade string, verification_status string, total_high_credit_limit float, application_type string, join_annual_income float, 
verification_status_joint string, ingest_date timestamp) using parquet location '/user/anil/lendingClubProject/cleansed/customersParquet' """)

""


In [6]:
spark.sql('select * from lendingClub_525.customers')

member_id,emp_title,emp_length,home_ownership,annual_income,address_state,address_zipcode,address_country,grade,sub_grade,verification_status,total_high_credit_limit,application_type,join_annual_income,verification_status_joint,ingest_date
5bad76b2473e8638c...,Funeral Director,3,RENT,56000.0,VA,221xx,USA,C,C2,Verified,61768.0,Individual,null,null,null
2651c8e4bc69338d8...,Technician,3,OWN,120000.0,OK,730xx,USA,A,A4,Not Verified,261126.0,Individual,null,null,null
2b3f2a8caa61c35eb...,International Fli...,10,MORTGAGE,60000.0,TX,773xx,USA,C,C3,Not Verified,144189.0,Individual,null,null,null
432ea84f5b9715a27...,Carpenter,5,RENT,12000.0,CA,945xx,USA,A,A5,Not Verified,16300.0,Individual,null,null,null
cfbb2e6dd37657fcc...,Office work,8,MORTGAGE,30000.0,FL,334xx,USA,A,A3,Not Verified,43000.0,Individual,null,null,null
d1946d1099882c06a...,Deli Manager,10,MORTGAGE,60000.0,NY,119xx,USA,B,B3,Not Verified,305570.0,Individual,null,null,null
3a8b3e793b034aa4b...,Hair stylist,1,RENT,30000.0,GA,303xx,USA,B,B3,Not Verified,56962.0,Individual,null,null,null
3576f3b052b25e27b...,RECEPTIONIST,10,RENT,102000.0,NJ,070xx,USA,B,B3,Verified,35435.0,Individual,null,null,null
a41b62fdeb37c6783...,Office Administrator,5,MORTGAGE,70000.0,FL,330xx,USA,D,D4,Source Verified,283460.0,Individual,null,null,null
efc9537c442236da0...,Pharmacist,1,RENT,80000.0,TN,379xx,USA,A,A5,Not Verified,64936.0,Individual,null,null,null


In [15]:
spark.sql("""create table if not exists lendingClub_525.loans (loan_id string, member_id string, loan_amount float, funded_amount float,
loan_term_years integer, interest_rate float, monthly_installment float, issue_date string,
loan_status string, loan_purpose string, loan_title string, ingest_date timestamp) stored as parquet location '/user/anil/lendingClubProject/cleansed/loansParquet' """)

""


In [7]:
spark.sql('select * from lendingClub_525.loans')

loan_id,member_id,loan_amount,funded_amount,loan_term_years,interest_rate,monthly_installment,issue_date,loan_status,loan_purpose,loan_title,ingest_date
59190291,0431f86978d2a85e9...,30000.0,30000.0,null,20.99,811.44,Oct-2015,Charged Off,credit_card,Credit card refin...,null
60972894,d238f9b04eee8c1c7...,18000.0,18000.0,null,16.99,447.25,Oct-2015,Charged Off,debt_consolidation,Debt consolidation,null
60960485,498dd5426f8f54cf6...,6300.0,6300.0,null,14.65,217.32,Oct-2015,Charged Off,debt_consolidation,Debt consolidation,null
61036633,60bcff6473e4dfedc...,10000.0,10000.0,null,12.69,225.95,Oct-2015,Fully Paid,debt_consolidation,Debt consolidation,null
60742072,a752b18765d069246...,8400.0,8400.0,null,7.26,260.37,Oct-2015,Fully Paid,credit_card,Credit card refin...,null
60823793,9eddecb0503ede6a3...,12325.0,12325.0,null,26.77,374.66,Oct-2015,Charged Off,debt_consolidation,Debt consolidation,null
60793669,986f35bcef677fd75...,35000.0,35000.0,null,13.33,802.29,Oct-2015,Charged Off,credit_card,Credit card refin...,null
60963101,1e068f117261f6a58...,10000.0,10000.0,null,9.17,318.79,Oct-2015,Fully Paid,debt_consolidation,Debt consolidation,null
60982977,9c082e773d5abee94...,18000.0,18000.0,null,9.99,382.36,Oct-2015,Current,debt_consolidation,Debt consolidation,null
60883787,c08faf281a34d5fd3...,7350.0,7350.0,null,19.99,273.12,Oct-2015,Fully Paid,vacation,Vacation,null


In [10]:
spark.sql("""create external table if not exists lendingClub_525.loans_repayments (loan_id_ string, total_principal_received float,
total_interest_received float,total_late_fee_received float,total_payment_received float,last_payment_amount float,
last_payment_date string,next_payment_date string,IngestedDateTime timestamp) stored as parquet 
location '/user/anil/lendingClubProject/cleansed/loansRepaymentParquet' """)

""


In [9]:
spark.sql('drop table lendingClub_525.loans_repayments')

""


In [11]:
spark.sql('select * from lendingClub_525.loans_repayments')

loan_id_,total_principal_received,total_interest_received,total_late_fee_received,total_payment_received,last_payment_amount,last_payment_date,next_payment_date,IngestedDateTime
68407277,3600.0,821.72,0.0,4421.724,122.67,null,null,2024-11-18 07:39:...
68355089,24700.0,979.66,0.0,25679.66,926.35,null,null,2024-11-18 07:39:...
68341763,20000.0,2705.92,0.0,22705.924,15813.3,null,null,2024-11-18 07:39:...
66310712,19102.35,12361.66,0.0,31464.01,829.9,Apr-2019,Apr-2019,2024-11-18 07:39:...
68476807,10400.0,1340.5,0.0,11740.5,10128.96,null,null,2024-11-18 07:39:...
68426831,11950.0,1758.95,0.0,13708.948,7653.56,null,null,2024-11-18 07:39:...
68476668,20000.0,1393.8,0.0,21393.8,15681.05,null,null,2024-11-18 07:39:...
67275481,20000.0,1538.51,0.0,21538.51,14618.23,null,null,2024-11-18 07:39:...
68466926,10000.0,998.97,0.0,10998.972,1814.48,null,null,2024-11-18 07:39:...
68616873,8000.0,939.58,0.0,8939.58,4996.24,null,null,2024-11-18 07:39:...


In [78]:
spark.sql("""create external table if not exists lendingClub_525.loans_defaulters_delinq (member_id string, delinq_2yrs integer, delinq_amnt float, mths_since_last_delinq integer) stored as parquet 
location '/user/anil/lendingClubProject/cleansed/NoLoansDefaultersParquet' """)

""


In [79]:
spark.sql('select * from lendingClub_525.loans_defaulters_delinq')

member_id,delinq_2yrs,delinq_amnt,mths_since_last_delinq
b59d80da191f5b573...,0,0.0,31
202d9f56ecb7c3bc9...,1,0.0,6
e5a140c0922b554b9...,0,0.0,47
e12aefc548f750777...,0,0.0,33
1b3a50d854fbbf97e...,1,0.0,21
9847d8c1e9d0b2084...,1,0.0,6
8340dbe1adea41fb4...,0,0.0,36
d4de0de3ab7d79ad4...,0,0.0,35
1d4e1ef4353b73c00...,0,0.0,30
6f196952e71277fd4...,4,0.0,5


In [80]:
spark.sql("""create external table if not exists lendingClub_525.loans_defaulters_detail_rec_enq ( member_id string, pub_rec integer, pub_rec_bankruptcies integer, inq_last_6mths integer) stored as parquet 
location '/user/anil/lendingClubProject/cleansed/LoansDefaultersParquet' """)

""


In [81]:
spark.sql('select * from lendingClub_525.loans_defaulters_detail_rec_enq')

member_id,pub_rec,pub_rec_bankruptcies,inq_last_6mths
b59d80da191f5b573...,0,0,1
202d9f56ecb7c3bc9...,0,0,0
e5a140c0922b554b9...,0,0,0
e12aefc548f750777...,0,0,0
1b3a50d854fbbf97e...,0,0,0
1c4329e5f17697127...,0,0,0
5026c86ad983175eb...,1,0,2
9847d8c1e9d0b2084...,2,0,0
8340dbe1adea41fb4...,0,0,0
d4de0de3ab7d79ad4...,0,0,0
